In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local", "ReadFromHDFS")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/15 19:04:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 1 - Load movies data into spark rdd tuple with (movieid, title, genres) format

rddMovies = sc.textFile("hdfs://localhost:9000/ml-latest-small/movies.csv")\
              .map(lambda line: tuple(line.split(",")))    

# to skip first line, 
headerLine = rddMovies.first()
moviesRdd = rddMovies.filter(lambda line: line != headerLine)

#print(rddMovies.take(1))

# get top 5 lines
print(moviesRdd.take(5))

22/05/15 19:04:24 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

[('1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'), ('2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'), ('3', 'Grumpier Old Men (1995)', 'Comedy|Romance'), ('4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'), ('5', 'Father of the Bride Part II (1995)', 'Comedy')]


In [3]:
genreRdd = moviesRdd.map (lambda stock: stock[2])
genreRdd.take(5)

22/05/15 19:04:25 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy']

In [4]:
#2 - Count number of movies based on Genres fields, 
# use Adventure, Romance, Comedy as specific genres, get the count of movies on each genres


genreCountRdd = genreRdd.map(lambda line: line.split("|"))\
                        .flatMap(lambda elements: elements)\
                        .map (lambda word: (word, 1))\
                        .reduceByKey(lambda acc, value: acc + value)

genreCountRdd.take(10)

[('Adventure', 990),
 ('Animation', 504),
 ('Children', 504),
 ('Comedy', 3047),
 ('Fantasy', 600),
 ('Romance', 1250),
 ('Drama', 3310),
 ('Action', 1499),
 ('Crime', 948),
 ('Thriller', 1443)]

In [5]:
# 3 - Load ratings.csv into tuple (userid, movieid, rating, timestamp) format

rddRatings = sc.textFile("hdfs://localhost:9000/ml-latest-small/ratings.csv")\
               .map(lambda line: tuple(line.split(",")))    

# to skip first line, 
#headerLine2 = rddRatings.first()
#ratingsRdd = rddRatings.filter(lambda line: line != headerLine2)



# get top 5 lines
rddRatings.take(5)



22/05/15 19:04:25 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

[('userId', 'movieId', 'rating', 'timestamp'),
 ('1', '1', '4.0', '964982703'),
 ('1', '3', '4.0', '964981247'),
 ('1', '6', '4.0', '964982224'),
 ('1', '47', '5.0', '964983815')]

In [6]:
# 4 - Remove the rating which is lower than 1.0 from the ratings.csv


# skip first line, 
headerLine2 = rddRatings.first()
RatingsRddC = rddRatings.filter(lambda line: line != headerLine2)\
                       .map(lambda t: (int(t[0]), int(t[1]), float(t[2]), int(t[3])))
                         

filteredRatingsRdd = RatingsRddC.filter(lambda t: t[2] >= 1.0)

#print(RatingsRddC.take(10)) 
filteredRatingsRdd.take(10)




#print(filteredRatingsRdd.count()) 





22/05/15 19:04:25 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

[(1, 1, 4.0, 964982703),
 (1, 3, 4.0, 964981247),
 (1, 6, 4.0, 964982224),
 (1, 47, 5.0, 964983815),
 (1, 50, 5.0, 964982931),
 (1, 70, 3.0, 964982400),
 (1, 101, 5.0, 964980868),
 (1, 110, 4.0, 964982176),
 (1, 151, 5.0, 964984041),
 (1, 157, 5.0, 964984100)]

In [7]:
# 5 - Use reduceByKey, count the ratings for the movies based on movieid, for example, if a movie received 10 ratings, 
#the output should be a tuple (movieId, 10)


# convert into (key,value) rdd  for reduceByKey
ratingsPairRdd = RatingsRddC.map (lambda t: (t[0], 1))
#print(ratingsPairRdd.take(10))

# get ratings count using for each movieId  ------ reduceByKey

ratingsCountRdd = ratingsPairRdd.reduceByKey(lambda acc, value: acc + value)
ratingsCountRdd.take(20)

[(1, 232),
 (2, 29),
 (3, 39),
 (4, 216),
 (5, 44),
 (6, 314),
 (7, 152),
 (8, 47),
 (9, 46),
 (10, 140),
 (11, 64),
 (12, 32),
 (13, 31),
 (14, 48),
 (15, 135),
 (16, 98),
 (17, 105),
 (18, 502),
 (19, 703),
 (20, 242)]

In [8]:
# write the result into text file in hdfs
# saveAsTextFile is an ACTION Method
# word-count-results is a folder, inside we will shall partition files

#ratingsCountRdd.saveAsTextFile("hdfs://localhost:9000/ratings-count-result")

In [9]:
### check flod file on github 

# 6 - Use Fold or FoldByKey, find the sum/or avg rating received by a movie in rating.csv


moviesKeyPairRdd = moviesRdd.map (lambda t: (t[0], t[1]))\
                             .map(lambda t: (int(t[0]), str(t[1])))

ratingsKeyPairRdd = RatingsRddC.map (lambda t: (t[1], t[2]))

#print(moviesKeyPairRdd.take(5))
#print(ratingsKeyPairRdd.take(5))

joinRdd = moviesKeyPairRdd.join(ratingsKeyPairRdd)\
                          .map(lambda t: t[1])


#print(joinRdd.take(20))


def add(acc, value):
    output = acc + value
    #print("acc", acc, "value", value, "output", output)
    return output

sumRatingByMovie = joinRdd.foldByKey(0, add).take(20)


sumRatingByMovie



[('Jumanji (1995)', 377.5),
 ('Waiting to Exhale (1995)', 16.5),
 ('Four Rooms (1995)', 74.0),
 ('Money Train (1995)', 37.5),
 ('Copycat (1995)', 116.0),
 ('Powder (1995)', 87.5),
 ('Persuasion (1995)', 46.5),
 ('Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 705.0),
 ('Babe (1995)', 467.5),
 ('It Takes Two (1995)', 8.5),
 ('Mighty Aphrodite (1995)', 97.0),
 ('"Big Green', 7.0),
 ('"Postman', 184.0),
 ('Two if by Sea (1996)', 11.0),
 ('Lawnmower Man 2: Beyond Cyberspace (1996)', 22.5),
 ('From Dusk Till Dawn (1996)', 193.0),
 ('Kicking and Screaming (1995)', 22.5),
 ('Bed of Roses (1996)', 32.0),
 ('"Crossing Guard', 10.0),
 ("Antonia's Line (Antonia) (1995)", 31.0)]

In [10]:
# 7 - Convert timestamp in rating.csv into Python date time object

import datetime

#print(RatingsRddC.take(5))


RatingsPythonDateTimeRddC= RatingsRddC.map(lambda t: (t[0], t[1], t[2], datetime.datetime.fromtimestamp(t[3]/1000)))

RatingsPythonDateTimeRddC.take(5)

22/05/15 19:04:27 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

[(1, 1, 4.0, datetime.datetime(1970, 1, 11, 22, 3, 2, 703000)),
 (1, 3, 4.0, datetime.datetime(1970, 1, 11, 22, 3, 1, 247000)),
 (1, 6, 4.0, datetime.datetime(1970, 1, 11, 22, 3, 2, 224000)),
 (1, 47, 5.0, datetime.datetime(1970, 1, 11, 22, 3, 3, 815000)),
 (1, 50, 5.0, datetime.datetime(1970, 1, 11, 22, 3, 2, 931000))]

In [11]:
# 8 - Join ratings and movies based on movie id, so that we could see movie title and genres for the given movie


moviesPairRdd = moviesRdd.map(lambda t: (t[0], t[1], t[2]))\
                         .map(lambda t: (int(t[0]), str(t[1]), str(t[2])))
#print(moviesPairRdd.take(5))

      
ratingsPairRdd = RatingsRddC.map(lambda t: (t[1], t[2]))
#print(ratingsPairRdd.take(5))
      
rddJoin = moviesPairRdd.join(ratingsPairRdd)
rddJoin.take(10)

[(2, ('Jumanji (1995)', 4.0)),
 (2, ('Jumanji (1995)', 4.0)),
 (2, ('Jumanji (1995)', 3.0)),
 (2, ('Jumanji (1995)', 3.0)),
 (2, ('Jumanji (1995)', 3.0)),
 (2, ('Jumanji (1995)', 3.5)),
 (2, ('Jumanji (1995)', 4.0)),
 (2, ('Jumanji (1995)', 4.5)),
 (2, ('Jumanji (1995)', 4.0)),
 (2, ('Jumanji (1995)', 2.5))]

In [12]:


# 9 - Use sort on question number 5, sort the movie rating count by descending order so that highest rated movie should be on the top


ratingsCountRdd.sortBy(lambda x: x[1], False).take(10)




[(414, 2698),
 (599, 2478),
 (474, 2108),
 (448, 1864),
 (274, 1346),
 (610, 1302),
 (68, 1260),
 (380, 1218),
 (606, 1115),
 (288, 1055)]